In [2]:
using DataFrames, CSV, JuMP, Gurobi, Random, LibPQ, Statistics

In [ ]:
#data = projections for each player

In [3]:
current_picks = Dict()
player_id = Dict()
qbs = Set()
rbs = Set()
wrs = Set()
tes = Set()

Set{Any}()

In [12]:
#add 1,2,3 to qbs
push!(qbs, 2)

Set{Any} with 2 elements:
  2
  1

In [14]:
in(1,qbs)+in(3,qbs)


1

In [ ]:
const GRB_ENV = Gurobi.Env(output_flag=0);

In [ ]:
function dynasty_draft()
    # create model
    model = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attribute(model, "TimeLimit", 300)

    # PARAMETERS
    T = 10 # teams
    R = 25 # roster spots
    P = 500 # num_players -- CHANGE
    min_qbs = 2
    max_qbs = 2
    min_rbs = 4
    max_rbs = 6
    min_wrs = 4
    max_wrs = 6
    min_tes = 2
    max_tes = 3

    # VARIABLES
    @variable(model, x[i = 1:T, j = 1:R] > 0, Int) # R roster spots each for N teams

    # OBJECTIVE FUNCTION - distribution for total expected value - maybe maximize the 25th percentile outcome?
    @objective(model, Max, sum(x[i,j,w] for i in 1:T, j in 1:R))

    # CONSTRAINTS
    # no player can be on two teams
    @constraint(model, [p = 1:P], sum(x[i,j]==p for i in 1:T, j in 1:R) <= 1)

    # position constraints for each team
    @constraint(model, [i = 1:T], sum(in(x[i,j], qbs) for j in 1:R) >= min_qbs)
    @constraint(model, [i = 1:T], sum(in(x[i,j], qbs) for j in 1:R) <= max_qbs)
    @constraint(model, [i = 1:T], sum(in(x[i,j], rbs) for j in 1:R) >= min_rbs)
    @constraint(model, [i = 1:T], sum(in(x[i,j], rbs) for j in 1:R) <= max_rbs)
    @constraint(model, [i = 1:T], sum(in(x[i,j], wrs) for j in 1:R) >= min_wrs)
    @constraint(model, [i = 1:T], sum(in(x[i,j], wrs) for j in 1:R) <= max_wrs)
    @constraint(model, [i = 1:T], sum(in(x[i,j], tes) for j in 1:R) >= min_tes)
    @constraint(model, [i = 1:T], sum(in(x[i,j], tes) for j in 1:R) <= max_tes)

    # already picked constraints
    @constraint(model, [i = 1:T], sum(x[i,j] for j in 1:R) == 16)


    # OPTIMIZE
    # solvetime = @elapsed optimize!(model)
    optimize!(model)


    return value.(x)
end

In [ ]:
function dynasty_pick(team)
    # create model
    model = Model(() -> Gurobi.Optimizer(GRB_ENV))
    set_optimizer_attribute(model, "TimeLimit", 300)

    # PARAMETERS
    T = 10 # teams
    R = 25 # roster spots
    P = 500 # num_players -- CHANGE
    min_qbs = 2
    max_qbs = 2
    min_rbs = 4
    max_rbs = 6
    min_wrs = 4
    max_wrs = 6
    min_tes = 2
    max_tes = 3

    # VARIABLES
    @variable(model, x[i = 1:T, j = 1:R] > 0, Int) # R roster spots each for N teams

    # OBJECTIVE FUNCTION - gotta figure out how to do this
    @objective(model, Max, sum(x[i,j,w] for i in 1:T, j in 1:R))

    # CONSTRAINTS
    # no player can be on two teams
    @constraint(model, [p = 1:P], sum(x[i,j]==p for i in 1:T, j in 1:R) <= 1)


    # position constraints for each team
    @constraint(model, [i = 1:T], sum(in(x[i,j], qbs) for j in 1:R) >= min_qbs)
    @constraint(model, [i = 1:T], sum(in(x[i,j], qbs) for j in 1:R) <= max_qbs)
    @constraint(model, [i = 1:T], sum(in(x[i,j], rbs) for j in 1:R) >= min_rbs)
    @constraint(model, [i = 1:T], sum(in(x[i,j], rbs) for j in 1:R) <= max_rbs)
    @constraint(model, [i = 1:T], sum(in(x[i,j], wrs) for j in 1:R) >= min_wrs)
    @constraint(model, [i = 1:T], sum(in(x[i,j], wrs) for j in 1:R) <= max_wrs)
    @constraint(model, [i = 1:T], sum(in(x[i,j], tes) for j in 1:R) >= min_tes)
    @constraint(model, [i = 1:T], sum(in(x[i,j], tes) for j in 1:R) <= max_tes)

    

    # each team can only play at most one game per week
    @constraint(model, [i = 1:T, w = 1:W], sum(x[i, j, w] + x[j, i, w] for j in 1:N) <= 1)

    
    


    # OPTIMIZE
    # solvetime = @elapsed optimize!(model)
    optimize!(model)


    return value.(x)
end